# \[Week5] 추천시스템  
## [캐글 데이터](https://www.kaggle.com/tmdb/tmdb-movie-metadata?select=tmdb_5000_movies.csv) tmdb_5000_movies.csv   
참고 사이트 : [[구름 Edu]](https://edu.goorm.io/learn/lecture/20671/%EB%94%A5%EB%9F%AC%EB%8B%9D-%EC%9D%B4%EB%A1%A0-%EB%B0%8F-%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%8B%A4%EC%8A%B5)

## 1. 자연어처리 - 문서 유사도 (용어 정리)

1. Bag of Words   

\[답변] 가방 속에 단어와 단어의 빈도수가 쌍으로 들어가있다. 가방을 흔들면 순서가 뒤죽박죽 섞인다.   
문장 유사도 구할 때, ml model의 입력값으로 사용 가능. 

1) 문장 간의 유사도 구하는 방법 

\[답변] 두 문장의 BoW의 값을 인덱스 별로 곱해서 더해준다. 값이 클수록 유사도가 높다. 

2) 단점

\[답변] sparsity; vector 차원이 커지면 0이 많아지고 메모리 비효율적이다.   
frequent words has more power   
ignoring word orders; 가방에 들어가있어서 출현 순서가 고려되지 않는다.    
out of vocabulary; 처음 보는 단어는 처리를 못한다. 

2. n-그램

\[답변] 연속적으로 n개의 token으로 구성된 것. 

**(예제) fine thank you**  
1) 1-gram

    1-1) Word level  

    [답변]  [fine, thank, you]

    1-2) Character level

    [답변]  [f, i, n, e,  , t, h, a, n, k,  , y, o, u]

2) 2-gram

    2-1) Word level

    [답변]  [fine thank, thank you]

    2-2) Character level 

    [답변] [fi, in, ne, e ,  t, th, ha, an, nk, k ,  y, yo, ou]

3) 3-gram

    3-1) Word level 

    [답변] [fine thank you]

    3-2) Character level 

    [답변] [fin, ine, ne , e t,  th, tha, han, ank, nk , k y,  yo, you]

4) n-gram 사용하는 이유?  

\[답변] BoW의 순서 무시 단점을 극복 가능. 다음 단어 예측, 오타 찾기

3. 점 A(3,4), 점 B(4,3)의 cosine 유사도를 구하시오.  

\[답변] A * B / (|A| * |B|) = ((3 * 4) + (4 * 3)) / (5 * 5) = cos(16.26°)

## 2. 추천시스템  
### 아래 코드에서 pass 부분을 지우고 그 자리에 코드 작성해주세요

### 0. 추천시스템 종류 정리

1. Content-based filtering  

\[답변] meta 정보를 feature로 사용한다. cold start 문제를 피할 수 있다. 

2. Collaborative filtering  

\[답변] 협력 필터링. user의 행동 양식, 패턴에 따라 추천한다. 수집된 정보 양이 많아야 효과가 좋다. 

    - user-based collaborative filtering  

    [답변] user끼리의 유사도를 측정한다. item이라는 vector로 표현됨. 비슷한 유저를 찾아 item 비교를 통해 추천

    - item-based collaborative filtering  

    [답변] item끼리의 유사도를 측정한다. user라는 vector로 표현됨. 비슷한 item을 추천

Content-based filtering을 해봅시다.  
보통은 장르를 기준으로 많이 하지만, 자연어처리에 좀 더 익숙해지기 위해서 줄거리를 바탕으로 해보겠습니다.  
Collaborative filtering도 한번 직접 해보세요. 재밌습니다:)

### 1. CSV 파일 불러와 DataFrame으로 저장

In [1]:
import pandas as pd
movies = pd.read_csv('tmdb_5000_movies.csv')           # tmdb_5000_movies.csv dataframe으로 읽어오기
movies = movies[["original_title", "overview"]]
movies["overview"] = movies["overview"].astype("str")
movies

,original_title,overview
0,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,Spectre,A cryptic message from Bond’s past sends him o...
3,The Dark Knight Rises,Following the death of District Attorney Harve...
4,John Carter,"John Carter is a war-weary, former military ca..."
...,...,...
4798,El Mariachi,El Mariachi just wants to play his guitar and ...
4799,Newlyweds,A newlywed couple's honeymoon is upended by th...
4800,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic..."
4801,Shanghai Calling,When ambitious New York attorney Sam is sent t...


### 2. 전처리  
출력 예시)

    original_title    0
    overview          3
    dtype: int64


In [2]:
# null 값 개수 확인
movies.isnull().sum()

original_title    0
overview          0
dtype: int64

1) 줄거리가 NaN인 **영화** drop

In [3]:
movies = movies.dropna(axis = 0)

2) "overview" column 모두 소문자로, 문자+숫자(\w)만 남기고 나머지는 띄어쓰기로 대체

In [4]:
import re
# 되도록 apply, lambda 사용할것!
movies['overview'] = movies['overview'].apply(lambda x: re.sub('[^\w]', ' ', x.lower()))
movies

,original_title,overview
0,Avatar,in the 22nd century a paraplegic marine is di...
1,Pirates of the Caribbean: At World's End,captain barbossa long believed to be dead ha...
2,Spectre,a cryptic message from bond s past sends him o...
3,The Dark Knight Rises,following the death of district attorney harve...
4,John Carter,john carter is a war weary former military ca...
...,...,...
4798,El Mariachi,el mariachi just wants to play his guitar and ...
4799,Newlyweds,a newlywed couple s honeymoon is upended by th...
4800,"Signed, Sealed, Delivered",signed sealed delivered introduces a dedic...
4801,Shanghai Calling,when ambitious new york attorney sam is sent t...


### 3. TF-IDF 계산

1) TF 란?  

\[답변] term frequency. 주어진 문서에서의 단어 출현 빈도. 특정 단어 출현빈도 / 모든 단어 출현 빈도. 

2) DF 란?  

\[답변] document frequency. 

3) IDF 란?   

\[답변] inverse document frequency. log(총 문장의 개수 / 그 단어가 출현한 문장의 개수)

라이브러리로 적용해보자...

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
# ngram_range=(1, 2) 는 단어를 1개 혹은 2개 연속으로 보겠다
tfidf_vec = TfidfVectorizer(ngram_range = (1, 2))                    # Vectorizer 생성
tfidf_matrix = tfidf_vec.fit_transform(movies['overview'])       # Vectorizer가 단어들을 학습

### 4. 영화 간 **cosine 유사도**  
\[출력예시]  

    ### COSINE Similarity ###
    [[1.         0.01513636 0.00614044 ... 0.01195144 0.00571986 0.00630023]
    [0.01513636 1.         0.01307772 ... 0.01768395 0.00997186 0.00666392]
    [0.00614044 0.01307772 1.         ... 0.01288316 0.00565125 0.00612543]
    ...
    [0.01195144 0.01768395 0.01288316 ... 1.         0.01532266 0.0089986 ]
    [0.00571986 0.00997186 0.00565125 ... 0.01532266 1.         0.01649368]
    [0.00630023 0.00666392 0.00612543 ... 0.0089986  0.01649368 1.        ]]
    [[   0 3603  634 ... 2852 3945 3386]
    ### 유사도 기준 index 정렬 ###
    [[   0 3603  634 ... 2852 3945 3386]
    [   1 2379 2542 ... 3487 3622 3945]
    [   2 1343 3161 ... 3534 3637 3082]
    ...
    [4797 4033  569 ... 2604 3685 1658]
    [4798 2017 1480 ... 2108 1658 4511]
    [4799 2586  868 ... 3987 2192  669]]



In [6]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
plot_similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)          # 줄거리 간 cosine 유사도 구하기 - tfidf_matrix 사용
print("### COSINE Similarity ###")
print(plot_similarity)
similar_index = np.argsort(-plot_similarity)         # 유사도 높은 순서대로 index 정렬
print("### 유사도 기준 index 정렬 ###") 
print(similar_index)

# https://wikidocs.net/76347

### COSINE Similarity ###
[[1.         0.01514413 0.00614504 ... 0.01195829 0.00572386 0.006304  ]
 [0.01514413 1.         0.01308527 ... 0.0176922  0.00997908 0.00666831]
 [0.00614504 0.01308527 1.         ... 0.01289    0.00565554 0.00612954]
 ...
 [0.01195829 0.0176922  0.01289    ... 1.         0.01532978 0.00900306]
 [0.00572386 0.00997908 0.00565554 ... 0.01532978 1.         0.01649947]
 [0.006304   0.00666831 0.00612954 ... 0.00900306 0.01649947 1.        ]]
### 유사도 기준 index 정렬 ###
[[   0 3604  634 ... 4140 2596 2669]
 [   1 2379 2542 ...  161 2656 4458]
 [   2 1343 3162 ... 4144 4148 4180]
 ...
 [4800 4034  569 ... 2853 4140 1038]
 [4801 2017 1480 ... 4140 4458 2108]
 [4802 2586  868 ... 3988 4513 3152]]


In [7]:
input_movie = "Avatar"          # data에 있는 영화의 제목을 넣어야 합니다.

movie_index = movies[movies.original_title == input_movie].index[0]    # input_movie에 해당하는 index 값 가져오기
similar_movies = similar_index[movie_index][1:11]                   # 유사도 상위 10개 index 가져오기
similar_movies
# 인덱스로 사용하기 위해서는 1차원으로 변형
similar_movies_index = [i for i in similar_movies]            # similar_movies 1차원 변형
display(movies.iloc[similar_movies_index])

,original_title,overview
3604,Apollo 18,officially apollo 17 was the last manned miss...
634,The Matrix,set in the 22nd century the matrix tells the ...
529,Tears of the Sun,navy seal lieutenant a k waters and his elite...
1013,Child 44,set in stalin era soviet union a disgraced mg...
775,Supernova,set in the 22nd century when a battered salva...
2130,The American,dispatched to a small italian town to await fu...
2628,Blood and Chocolate,a young teenage werewolf is torn between honor...
1341,Obitaemyy Ostrov,on the threshold of 22nd century furrowing th...
2808,The Exorcist,12 year old regan macneil begins to adapt an e...
311,The Adventures of Pluto Nash,the year is 2087 the setting is the moon plu...


In [8]:
# https://wikidocs.net/24603
input_movie = "The Dark Knight Rises"  

movie_index = movies[movies.original_title == input_movie].index[0]    # input_movie에 해당하는 index 값 가져오기
similar_movies = similar_index[movie_index][1:11]                   # 유사도 상위 10개 index 가져오기
similar_movies
# 인덱스로 사용하기 위해서는 1차원으로 변형
similar_movies_index = [i for i in similar_movies]            # similar_movies 1차원 변형
display(movies.iloc[similar_movies_index])

,original_title,overview
299,Batman Forever,the dark knight of gotham city confronts a das...
65,The Dark Knight,batman raises the stakes in his war on crime ...
1359,Batman,the dark knight of gotham city begins his war ...
428,Batman Returns,having defeated the joker batman now faces th...
2507,Slow Burn,a district attorney ray liotta is involved i...
119,Batman Begins,driven by tragedy billionaire bruce wayne ded...
3854,"Batman: The Dark Knight Returns, Part 2",batman has stopped the reign of terror that th...
1181,JFK,new orleans district attorney jim garrison dis...
210,Batman & Robin,along with crime fighting partner robin and ne...
4649,Shotgun Stories,shotgun stories tracks a feud that erupts betw...


3주동안의 자연어처리 스터디가 끝났습니다. 다음주부터는 이미지처리 스터디가 시작됩니다.  
짧은 기간이라 많은 부분을 다루지는 못했지만, 추후에 프로젝트를 진행하면서 멤버들끼리 딥러닝 등 심화 부분을 공부하시면 더 좋을 것 같습니다.  
중간에 시험기간도 겹쳐서 바쁜 와중에도 과제 잘 해주셔서 감사합니다!